# Lives Saved Estimates with Python MINIMOD

In [1]:
import pandas as pd
import minimod
from glob import glob
from pathlib import Path
import re

Using Python-MIP package version 1.7.3


In [2]:
# Folder with data
data_folder = Path('data_files')

## Processing Data

In [11]:
# get just the lives saved .xlsx files 
data_files = data_folder.glob(r'*Lives *')

In [12]:
list(data_files)

[PosixPath('data_files/APril22_VitaminA AND ZInc Combined Lives saved.xlsx'),
 PosixPath('data_files/APril22_Vitamin A alone Lives saved.xlsx')]

In [13]:
# Create dictionary for all files
all_dfs = {}

# Read in data
for d in data_files:
    df = pd.read_excel(d)
    all_dfs[d.stem] = df

In [27]:
# Now rename columns:
# - intervention
# - region
# Then set index to intervention and region, and stack time
final_dfs = []

for df in all_dfs:

    stacked_df = all_dfs[df].set_index()


Index(['Zinc interventions ',              'Region',                  2012,
                        2013,                  2014,                  2015,
                        2016,                  2017,                  2018,
                        2019,                  2020,                  2021,
                        2022,                  2023,                  2024,
                        2025,                  2026,                  2027,
                        2028,                  2029,                  2030,
                        2031,                  2032,                  2033,
                        2034,                  2035],
      dtype='object')
Index(['Vitamin A, AND Zinc intervention ',
                                    'Zone',
                                      2012,
                                      2013,
                                      2014,
                                      2015,
                                      2016,
        

## Summary Statistics